In [ ]:
#This is the notebook for tools_project

In [1]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [2]:
# pandas library to create dataframe
import pandas as pd
import numpy as np

In [4]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat now ? Ramen
Where are you ? New York


'https://www.yelp.com/search?find_desc=Ramen&find_loc=New York'

In [5]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [6]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [7]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [8]:
businesses

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="61undF7jnmIEldA6cBlfcQ" href="/adredir?ad_business_id=-Q8MYIWoE4EsGQtudIOY0w&amp;campaign_id=PLn3A76M7MyMA9deIl4atQ&amp;click_origin=search_results&amp;placement=above_search&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fbua-thai-ramen-and-robata-grill-new-york&amp;request_id=f9536fe4cd959630&amp;signature=049898ff9b1998ec2ac452d9bfccb7e3aa268778c4708e06d3cf36479c560d18&amp;slot=0"><span>Bua Thai Ramen &amp; Robata Grill</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="oyXabgScXfAcX5TnyRJHpw" href="/biz/menkoi-sato-new-york-2?osq=Ramen"><span>Menkoi Sato</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="WXGLNKrunAEuV0QVBYwiKQ" href="/biz/totto-ramen-new-york-12?osq=Ramen"><span>Totto Ramen</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercar

In [10]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this particular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [11]:
print(name,link)

Menkoi Sato https://www.yelp.com/biz/menkoi-sato-new-york-2?osq=Ramen


In [13]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [14]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [15]:
json_map

'{"serviceAreas": [], "moMapPossible": true, "scrollwheelZoom": false, "zoomControlPosition": "top_right", "minZoomlevel": null, "isFullBleed": false, "maxZoomlevel": null, "zoom": 15, "library": "google", "fitToGeobox": false, "hoods": [], "adPinColor": null, "markers": [{"location": null, "key": "directions_marker", "icon": {"name": "directions", "anchorOffset": [12, 32], "activeOrigin": [0, 0], "scaledSize": [24, 32], "regularUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/79f63ebc20db/assets/img/directions@2x.png", "size": [24, 32], "activeUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/7249ab345ac8/assets/img/directions_highlighted@2x.png", "regularOrigin": [0, 0]}}, {"resourceType": "business", "url": "/biz/menkoi-sato-new-york-2", "resourceId": "s2ii7iNxb0-hGVxHf0yexw", "shouldOpenInNewTab": false, "location": {"latitude": 40.7316536618558, "longitude": -74.0017022376845}, "key": "starred_business", "hovercardId": "oyXabgScXfAcX5TnyRJHpw", "icon": {"n

In [16]:
print(latitude,longitude)

40.7316536618558 -74.0017022376845


In [17]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [18]:
category

'Ramen'

In [19]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [20]:
print(price_range)

$$


In [21]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [22]:
print(rating_value,review_count)

4.5 72


In [23]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [24]:
print(phone_number)

(212) 727-1050


In [25]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [26]:
print(open_)

Open now


In [30]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

'7 Cornelia StNew York, NY 10014'

In [36]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

In [37]:
#want to put all infomations we want to get from one business into one funtion
#input: url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
    dict_map = json.loads(json_map)
    latitude = dict_map["markers"][1]["location"]["latitude"]
    longitude = dict_map["markers"][1]["location"]["longitude"]

    category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    ratings = results_business_page.find("div", itemprop = 'aggregateRating')
    rating_value = ratings.find("meta").get('content')
    review_count = ratings.find("span").get_text()
    
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = None
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
        
    return(addr,latitude,longitude,category,price_range,rating_value,review_count,phone_number,open_,web)
    

In [38]:
#test the function on cafe mogador
restaurant_url = "https://www.yelp.com/biz/cafe-mogador-brooklyn?osq=Restaurants"
get_info(restaurant_url)

('133 Wythe AveBrooklyn, NY 11211',
 40.719747,
 -73.959993,
 'Moroccan',
 '$$',
 '4.5',
 '1004',
 '(718) 486-9222',
 'Open now',
 'cafemogador.com/williamsburg')

In [39]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants
link_list = []

for i in range(1,len(businesses)):
    business = businesses[i]
    link = 'https://www.yelp.com' + business.get('href')
    link_list.append(link)

In [40]:
link_list

['https://www.yelp.com/biz/menkoi-sato-new-york-2?osq=Ramen',
 'https://www.yelp.com/biz/totto-ramen-new-york-12?osq=Ramen',
 'https://www.yelp.com/biz/ramen-hood-new-york?osq=Ramen',
 'https://www.yelp.com/biz/teppen-ramen-new-york-2?osq=Ramen',
 'https://www.yelp.com/biz/ramen-spot-brooklyn-4?osq=Ramen',
 'https://www.yelp.com/biz/ippudo-ny-new-york-7?osq=Ramen',
 'https://www.yelp.com/biz/tenichi-ramen-cobble-hill-brooklyn?osq=Ramen',
 'https://www.yelp.com/biz/ramen-mafia-brooklyn?osq=Ramen',
 'https://www.yelp.com/biz/oki-pok%C3%A9-and-ramen-new-york?osq=Ramen',
 'https://www.yelp.com/biz/ippudo-westside-new-york?osq=Ramen',
 'https://www.yelp.com/biz/momosan-ramen-and-sake-new-york-2?osq=Ramen',
 'https://www.yelp.com/biz/ramen-ya-new-york-5?osq=Ramen',
 'https://www.yelp.com/biz/tonchin-new-york-new-york?osq=Ramen',
 'https://www.yelp.com/biz/ramen-lab-new-york-2?osq=Ramen',
 'https://www.yelp.com/biz/minca-new-york?osq=Ramen',
 'https://www.yelp.com/biz/three-bowls-noodle-house

In [41]:
#Create a list of restaurant names
name_list=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    name_list.append(name)

In [42]:
name_list

['Menkoi Sato',
 'Totto Ramen',
 'Ramen Hood',
 'Teppen Ramen',
 'Ramen Spot',
 'Ippudo NY',
 'Tenichi Ramen Cobble Hill',
 'Ramen Mafia',
 'Oki Poké & Ramen',
 'Ippudo Westside',
 'Momosan Ramen & Sake',
 'Ramen-Ya',
 'TONCHIN NEW YORK',
 'Ramen Lab',
 'Minca',
 'Three Bowls Noodle House',
 'Hashi Ramen & Izakaya',
 'Chuko',
 'Jun-Men Ramen Bar',
 'Zundo-Ya',
 'Mu Ramen',
 'Shinshi Ramen',
 'Tamashii Ramen',
 'Nishida Sho-ten',
 'HinoMaru Ramen',
 'Mei Jin Ramen',
 'Kogane Ramen',
 'Totto Ramen - Midtown East',
 'Jin Sushi & Ramen',
 'Shuya Cafe de Ramen']

In [43]:
#Create a list of all restaurants' information
info_list=[]

for i in range(len(link_list)):
    info_list.append(get_info(link_list[i]))

In [46]:
info_list

[('7 Cornelia StNew York, NY 10014',
  40.7316536618558,
  -74.0017022376845,
  'Ramen',
  '$$',
  '4.5',
  '72',
  '(212) 727-1050',
  'Open now',
  None),
 ('366 W 52nd StNew York, NY 10019',
  40.7643285462369,
  -73.9876558773155,
  'Ramen',
  '$$',
  '4.0',
  '5001',
  '(212) 582-0052',
  'Open now',
  'tottoramen.com'),
 ("62 Spring StChef's Club CounterNew York, NY 10012",
  40.7223112701284,
  -73.9973774072879,
  'Ramen',
  '$$',
  '4.5',
  '31',
  '',
  None,
  'chefsclubcounter.com'),
 ('159 E 55th StNew York, NY 10022',
  40.7592651,
  -73.9688038,
  'Ramen',
  '$$',
  '4.5',
  '222',
  '(212) 355-2974',
  'Open now',
  'teppenramen.com'),
 ('760 Manhattan AveBrooklyn, NY 11222',
  40.7268634,
  -73.9523341,
  'Ramen',
  '$$',
  '4.5',
  '23',
  '(718) 809-2527',
  'Open now',
  None),
 ('65 4th AveNew York, NY 10003',
  40.730948,
  -73.990287,
  'Ramen',
  '$$',
  '4.0',
  '9559',
  '',
  'Open now',
  'ippudony.com'),
 ('116 Smith StBrooklyn, NY 11201',
  40.6878049,
  -

In [44]:
#Create a dictionary showing the information of each restaurant
business_info={}

for i in range(len(name_list)):
    business_info[name_list[i]] = info_list[i]        

In [45]:
business_info

{'Menkoi Sato': ('7 Cornelia StNew York, NY 10014',
  40.7316536618558,
  -74.0017022376845,
  'Ramen',
  '$$',
  '4.5',
  '72',
  '(212) 727-1050',
  'Open now',
  None),
 'Totto Ramen': ('366 W 52nd StNew York, NY 10019',
  40.7643285462369,
  -73.9876558773155,
  'Ramen',
  '$$',
  '4.0',
  '5001',
  '(212) 582-0052',
  'Open now',
  'tottoramen.com'),
 'Ramen Hood': ("62 Spring StChef's Club CounterNew York, NY 10012",
  40.7223112701284,
  -73.9973774072879,
  'Ramen',
  '$$',
  '4.5',
  '31',
  '',
  None,
  'chefsclubcounter.com'),
 'Teppen Ramen': ('159 E 55th StNew York, NY 10022',
  40.7592651,
  -73.9688038,
  'Ramen',
  '$$',
  '4.5',
  '222',
  '(212) 355-2974',
  'Open now',
  'teppenramen.com'),
 'Ramen Spot': ('760 Manhattan AveBrooklyn, NY 11222',
  40.7268634,
  -73.9523341,
  'Ramen',
  '$$',
  '4.5',
  '23',
  '(718) 809-2527',
  'Open now',
  None),
 'Ippudo NY': ('65 4th AveNew York, NY 10003',
  40.730948,
  -73.990287,
  'Ramen',
  '$$',
  '4.0',
  '9559',
  '',

In [64]:
def get_businesses(limit = 200):
    
    keyword = input('What do you want to eat ? ')
    address = input('Where are you ? ')
        
    
    url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
    print(url)
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")
    i = 0
    for business in businesses:
        name = business.find('span').get_text()
        link = 'https://www.yelp.com' + business.get('href')
        info_list2 = get_info(link)
        i += 1


In [65]:
get_businesses()

What do you want to eat ? burger
Where are you ? boston
https://www.yelp.com/search?find_desc=burger&find_loc=boston


AttributeError: 'NoneType' object has no attribute 'get'